# Data migration

### Setup

In [1]:
import pymysql
import pandas as pd
from paramiko import SSHClient
from sshtunnel import SSHTunnelForwarder
from os.path import expanduser
import json
import requests
from tqdm import tqdm

In [9]:
ENV = "local" # or 'dev' / 'prod'

In [10]:
sql_hostname = 'localhost'
sql_username = 'root'
sql_password = 'q1w2e3r4t5'
sql_main_database = 'wpdikdas'
sql_port = 3306
ssh_host = '10.1.9.121'
ssh_password = "vendor#1234"
ssh_user = 'vendor'
ssh_port = 2299

### SSH

In [11]:
if ENV == "prod":
    tunnel = SSHTunnelForwarder(
            (ssh_host, ssh_port),
            ssh_username=ssh_user,
            ssh_password=ssh_password,
            remote_bind_address=(sql_hostname, sql_port))

    tunnel.start()

In [12]:
if ENV == "prod":
    conn = pymysql.connect(
        host=sql_hostname, 
        user=sql_username,
        passwd=sql_password, 
        db=sql_main_database,
        port=tunnel.local_bind_port
    )

### Local DB

In [13]:
if ENV == "local":
    sql_password = 'frostysnow_98'

In [14]:
if ENV == "local":
    conn = pymysql.connect(
        host=sql_hostname, 
        user=sql_username,
        passwd=sql_password, 
        db=sql_main_database,
    )

## Get Data

In [15]:
def get_and_store_json(query, json_filename): 
    data = pd.read_sql_query(query, conn)
    data.T.to_json(json_filename)
    return data

def get(query): 
    data = pd.read_sql_query(query, conn)
    return data

## Migrate Post

### Post: post

In [16]:
q = '''
    SELECT p.*, a.guid AS cover_url, v.post_views_total 
    FROM wpdikdas.wpdikdas_posts p 
    JOIN wpdikdas.wpdikdas_postmeta m 
    JOIN wpdikdas.wpdikdas_posts a
    JOIN (
		SELECT post_id, sum(post_views_total) AS post_views_total 
        FROM wpdikdas.wpdikdas_post_views_realtime 
        GROUP BY post_id
	) v
    ON 
    p.ID = m.post_id AND 
    m.meta_value = a.ID AND
    p.ID = v.post_id
    
    WHERE 
    p.post_type = 'post' AND
    m.meta_key = '_thumbnail_id' AND 
    a.post_type = 'attachment'
'''

In [17]:
post = get_and_store_json(q, "post.json")

In [18]:
post.head()

,ID,post_author,post_date,post_date_gmt,post_content,post_title,post_excerpt,post_status,comment_status,ping_status,...,post_modified_gmt,post_content_filtered,post_parent,guid,menu_order,post_type,post_mime_type,comment_count,cover_url,post_views_total
0,84,1,2014-01-19 13:20:37,2014-01-19 06:20:37,"<p style=""text-align: justify;"">Harianjogja.co...","Pramuka Harus Kreatif, agar Disukai Anak",,publish,open,open,...,2014-02-24 05:47:33,,0,http://dikdas.kemdikbud.go.id/?p=84,0,post,,0,http://dikdas.kemdikbud.go.id/wp-content/uploa...,1269.0
1,139,1,2014-02-06 17:02:26,2014-02-06 10:02:26,"<p style=""text-align: center;"">Oleh <b>Prof. S...",Kurikulum Pendidikan Haruslah Memberi Tantanga...,,publish,open,open,...,2014-02-06 10:03:27,,0,http://dikdas.kemdikbud.go.id/?p=139,0,post,,0,http://dikdas.kemdikbud.go.id/wp-content/uploa...,90527.0
2,179,1,2014-01-08 16:44:47,2014-01-08 09:44:47,"<a href=""http://dikdas.kemdikbud.go.id/wp-cont...","DAK Dikdas Rp 6,5 T",,publish,open,open,...,2014-02-13 06:07:53,,0,http://dikdas.kemdikbud.go.id/?p=179,0,post,,0,http://dikdas.kemdikbud.go.id/wp-content/uploa...,2730.0
3,190,1,2014-02-05 16:17:57,2014-02-05 09:17:57,"<a href=""http://dikdas.kemdikbud.go.id/wp-cont...",Perbaikan Sekolah di Kota Bekasi Rp 60 Miliar,,publish,open,open,...,2014-02-05 09:17:57,,0,http://dikdas.kemdikbud.go.id/?p=190,0,post,,0,http://dikdas.kemdikbud.go.id/wp-content/uploa...,755.0
4,193,1,2014-02-05 16:22:44,2014-02-05 09:22:44,"<a href=""http://dikdas.kemdikbud.go.id/wp-cont...",Indonesia Jadi Tamu Kehormatan di Frankfurt Bo...,,publish,open,open,...,2014-02-05 09:22:44,,0,http://dikdas.kemdikbud.go.id/?p=193,0,post,,0,http://dikdas.kemdikbud.go.id/wp-content/uploa...,680.0


In [19]:
post.shape

(2241, 25)

In [20]:
json_post = json.loads(post.T.to_json())

### Post: revision

In [21]:
q = '''
    SELECT * FROM wpdikdas_posts
    WHERE post_type = 'revision'
'''

In [22]:
revision = get_and_store_json(q, "post_revision.json")

In [23]:
revision.head()

,ID,post_author,post_date,post_date_gmt,post_content,post_title,post_excerpt,post_status,comment_status,ping_status,...,pinged,post_modified,post_modified_gmt,post_content_filtered,post_parent,guid,menu_order,post_type,post_mime_type,comment_count
0,5,1,2014-02-01 10:40:52,2014-02-01 03:40:52,,Visi dan Misi,,inherit,open,open,...,,2014-02-01 10:40:52,2014-02-01 03:40:52,,4,http://dikdas.kemdikbud.go.id/baru/?p=5,0,revision,,0
1,7,1,2014-02-01 10:41:20,2014-02-01 03:41:20,,Struktur Organisasi,,inherit,open,open,...,,2014-02-01 10:41:20,2014-02-01 03:41:20,,6,http://dikdas.kemdikbud.go.id/baru/?p=7,0,revision,,0
2,11,1,2014-02-01 10:42:06,2014-02-01 03:42:06,,DAPODIK,,inherit,open,open,...,,2014-02-01 10:42:06,2014-02-01 03:42:06,,10,http://dikdas.kemdikbud.go.id/baru/?p=11,0,revision,,0
3,13,1,2014-02-01 10:42:53,2014-02-01 03:42:53,,Hubungi Kami,,inherit,open,open,...,,2014-02-01 10:42:53,2014-02-01 03:42:53,,12,http://dikdas.kemdikbud.go.id/baru/?p=13,0,revision,,0
4,30,1,2014-02-03 09:12:58,2014-02-03 02:12:58,"Gedung E Lantai 5 Jln. Jendral Sudirman,\nSena...",Hubungi Kami,,inherit,open,open,...,,2014-02-03 09:12:58,2014-02-03 02:12:58,,12,http://dikdas.kemdikbud.go.id/12-autosave-v1/,0,revision,,0


In [24]:
json_revision = json.loads(revision.T.to_json())

## Merge both

In [25]:
merged_post = {}
for k in json_post:
    post = json_post[k]
    merged_post[post["ID"]] = post

In [26]:
for k in json_revision:
    rev = json_revision[k]
    
    if rev["post_parent"] in merged_post:
        post = merged_post[rev["post_parent"]]
        merged_post[rev["post_parent"]] = {**post, **rev}

## Construct legit post 

In [27]:
from slugify import slugify
from datetime import datetime, timezone
import re

# map dikdas userid to strapi userid
userIDMapping = {
    1: 3,
    6: 4,
    7: 5,
    8: 6,
    9: 7,
}

def migrate_domain(text):
    text = re.sub('http://dikdas.kemdikbud.go.id', "{{MIGRATED_IMAGE_BASE_URL}}", text)
    text = re.sub('http://dikdasmen.kemdikbud.go.id', "{{MIGRATED_IMAGE_BASE_URL}}", text)
    
    return text

def get_time(unix_time):
    return datetime.fromtimestamp(unix_time).isoformat()

In [28]:
def migrate_post(post):
    return {
        "titleID": post['post_title'],
        "slug": slugify(post['post_title']),
        "visitCount": post["post_views_total"],
        "created_by": {
            "id": userIDMapping[post["post_author"]]
        },
        "updated_by": {
            "id": userIDMapping[post["post_author"]]
        },
        "isApproved": True,
        "publishDate": get_time(post["post_date"]/1000),
        "coverURL": migrate_domain(post["cover_url"]),
        "body": {
            "bodyID": migrate_domain(post["post_content"])
        },
        "migrated": True,
    }

## Send Migration Request

In [37]:
url = 'http://localhost:1337/news-articles'

In [ ]:
url = 'https://kemendikbud.herokuapp.com/news-articles'

## Becareful!

In [38]:
c = 20

for post_id in tqdm(merged_post):
    old_post = merged_post[post_id]
    new_post = migrate_post(old_post)

    requests.post(url, json=new_post)
    
    c -= 1
    if c == 0:
        break

  1%|          | 18/2241 [00:00<01:02, 35.33it/s]


## Post joined with its thumbnail

```
SELECT p.*, a.guid as cover_url 
FROM wpdikdas.wpdikdas_posts p 
JOIN wpdikdas.wpdikdas_postmeta m 
JOIN wpdikdas.wpdikdas_posts a
ON p.ID = m.post_id and m.meta_value = a.ID
WHERE 
p.post_type = 'post' AND
m.meta_key = '_thumbnail_id' AND 
a.post_type = 'attachment'
LIMIT 10;
```

there are 463 post attachment without post_parent out of 2925 images

```
SELECT count(*) FROM wpdikdas.wpdikdas_posts
where post_type = "attachment" and post_parent = "" and post_mime_type = "image/jpeg"
```

```
SELECT count(*) FROM wpdikdas.wpdikdas_posts
where post_mime_type = "image/jpeg"
```

## Migrate Gallery

In [29]:
q = '''
    SELECT guid
    FROM wpdikdas.wpdikdas_posts 
    WHERE 
    post_type = 'attachment' AND
    post_mime_type = 'image/jpeg'
    ORDER BY post_date ASC
'''

In [30]:
images = get_and_store_json(q, "images.json")

In [31]:
images.shape

(2925, 1)

In [32]:
images.head()

,guid
0,http://dikdas.kemdikbud.go.id/wp-content/uploa...
1,http://dikdas.kemdikbud.go.id/wp-content/uploa...
2,http://dikdas.kemdikbud.go.id/wp-content/uploa...
3,http://dikdas.kemdikbud.go.id/wp-content/uploa...
4,http://dikdas.kemdikbud.go.id/wp-content/uploa...


In [33]:
images = list(images["guid"])

In [34]:
gallery_pictures = []

temp = []
for i in range(len(images)):
    image_url = migrate_domain(images[i])
    temp.append(image_url)
    
    if (i + 1) % 50 == 0:
        gallery_pictures.append(temp)

## Create Gallery

In [35]:
import random
import string

def get_random_slug(k = 8):
    return ''.join(random.choices(string.ascii_lowercase + string.digits, k=k))

In [36]:
def migrate_gallery(gallery_id, pictures):
    return {
        "titleID": "Gallery {:02d}".format(gallery_id),
        "titleEN": "Gallery {:02d}".format(gallery_id),
        "subtitle": {
            "subtitleID": "Koleksi gambar dan foto",
            "subtitleEN": "Picture and images collection",
        },
        "slug": "gallery-{:02d}-{}".format(gallery_id, get_random_slug(8)),
        "coverURL": pictures[0],
        "pictureURLs": "\n".join(pictures),
        "created_by": {
            "id": 3,
        },
        "updated_by": {
            "id": 3,
        },
        "isActive": True,
        "migrated": True,
    }

## Send Migration Request

In [54]:
url = 'http://localhost:1337/galleries'

In [ ]:
url = 'https://kemendikbud.herokuapp.com/galleries'

In [55]:
c = 10

for i in tqdm(range(len(gallery_pictures))):
    new_gallery = migrate_gallery(i, gallery_pictures[i])
    requests.post(url, json=new_gallery)
    
    c -= 1
    if c == 0:
        break

 16%|█▌        | 9/58 [00:01<00:06,  7.23it/s]


## Attachments

In [37]:
q = '''
    SELECT guid
    FROM wpdikdas.wpdikdas_posts 
    WHERE 
    post_type = 'attachment' AND
    post_mime_type != 'image/jpeg'
    ORDER BY post_date ASC
'''

In [38]:
attachments = get_and_store_json(q, "attachments.json")

In [39]:
attachments.shape

(1353, 1)